In [1]:
import duckdb
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path 
# df = pd.read_parquet('datasets/competencia_01_aum.parquet')


# tabla = pa.Table.from_pandas(df)

# # write df as a parquet dataset
# ds.write_dataset(tabla, base_dir = "datasets/competencia_01_aumpy",
#                   format="parquet",
#                  partitioning= ["foto_mes"],
#                  max_rows_per_file = 42000, max_rows_per_group = 42000,
#                  existing_data_behavior = "overwrite_or_ignore")

In [3]:

def preparar_data(dbname, mes_train, mes_test, drop_cols=None, sampling = 1):

        #%%
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
    from sklearn.ensemble import RandomForestClassifier
    import lightgbm as lgb
    import optuna
    from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour
    from time import time
    import pickle
    import os
    from dotenv import load_dotenv
    import urllib
    import pyarrow.dataset as ds
    import lgbm_globales
    import duckdb
    from pathlib import Path


    print('Preparando data...')

    root_path = Path(__file__).parent.parent.resolve()

    root_path = root_path.as_posix() + '/'

    # Connect to DuckDB database
    con = duckdb.connect(root_path+'duckdb/'+ dbname + ".duckdb")       
    
    query = (f"""
            SELECT *
            FROM {dbname}
            JOIN {dbname}_lags
            ON {dbname}.foto_mes = {dbname}_lags.foto_mes
            AND {dbname}.numero_de_cliente = {dbname}_lags.numero_de_cliente
            AND {dbname}.clase_ternaria = {dbname}_lags.clase_ternaria;""")

    # Execute the join query and fetch the result as a Pandas DataFrame
    data = con.execute(query).fetchdf()

    # data = pd.read_parquet(dataset_path + dataset_file)

    data['clase_peso'] = 1.0

    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

    data['clase_binaria1'] = 0
    data['clase_binaria2'] = 0
    data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
    data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

    data = data.loc[data['foto_mes'].isin(mes_train + mes_test)]

    data_continua = data[data['clase_ternaria'] == 'CONTINUA']

    data_continua = data_continua.groupby('foto_mes').sample(frac=sampling, random_state=42)

    data = pd.concat[data_continua, data[data['clase_ternaria'] != 'CONTINUA']]

    print(data.head())

    del data_continua

    train_data = data.loc[data['foto_mes'].isin(mes_train)]
    test_data = data[data['foto_mes'] == mes_test]

    X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_train_binaria1 = train_data['clase_binaria1']
    y_train_binaria2 = train_data['clase_binaria2']
    w_train = train_data['clase_peso']

    X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_test_binaria1 = test_data['clase_binaria1']
    y_test_class = test_data['clase_ternaria']
    w_test = test_data['clase_peso']

    print('Data preparada')

    return X_train, y_train_binaria1, y_train_binaria2, w_train, X_test, y_test_class, y_test_binaria1, w_test



In [2]:

dbname = "compe_02"

In [3]:


# Connect to DuckDB database
con = duckdb.connect('duckdb/'+ dbname + ".duckdb")

In [9]:
query = (f"""
        SELECT foto_mes, count(*)
        FROM {dbname}
        GROUP BY foto_mes;""")

# Execute the join query and fetch the result as a Pandas DataFrame
data = con.execute(query).fetchdf()


In [10]:
data

,foto_mes,count_star()
0,201901,124752
1,201902,125799
2,201903,126436
3,201904,126996
4,201905,127659
5,201906,129186
6,201907,130724
7,201908,132664
8,201909,134314
9,201910,136682


In [26]:
query = (f"""
        SELECT *
        FROM {dbname}
        JOIN {dbname}_lags
        ON {dbname}.foto_mes = {dbname}_lags.foto_mes
        AND {dbname}.numero_de_cliente = {dbname}_lags.numero_de_cliente
        AND {dbname}.clase_ternaria = {dbname}_lags.clase_ternaria
        LIMIT 500000;""")

# Execute the join query and fetch the result as a Pandas DataFrame
data = con.execute(query).fetchdf()

data

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,tc_proporcion_consumo_pesos,tc_proporcion_consumo_dolares,tc_proporcion_consumo_total_limite_total_cubierto,tc_proporcion_pago_pesos,tc_proporcion_pago_dolares,tc_proporcion_adelanto_pesos,tc_proporcion_adelanto_dolares,numero_de_cliente_1,foto_mes_1,clase_ternaria_1
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,1.0,0.000129,0.061601,-6.188851,0.000366,NaN,NaN,249221109,201901,CONTINUA
1,249221109,201902,1,0,1,59,277,3800.93,46605.81,2104.71,...,1.0,0.000262,0.051877,-4.021923,0.000065,NaN,NaN,249221109,201902,CONTINUA
2,249221109,201903,1,0,1,59,278,4102.67,45348.80,2492.53,...,1.0,0.000176,0.061586,-2.142776,0.001116,NaN,NaN,249221109,201903,CONTINUA
3,249221109,201904,1,0,1,60,279,4374.99,44872.11,3231.46,...,1.0,0.000053,0.170511,-4.470767,0.000000,NaN,NaN,249221109,201904,CONTINUA
4,249221109,201907,1,0,1,60,282,3341.93,44801.06,2727.38,...,1.0,0.000122,0.076126,-4.202987,0.000000,NaN,NaN,249221109,201907,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,295641952,201910,1,0,1,61,251,0.00,0.00,0.00,...,1.0,0.001411,0.012774,NaN,NaN,NaN,NaN,295641952,201910,CONTINUA
499996,295641952,201911,1,0,1,61,252,6841.44,61155.08,2439.95,...,1.0,0.000452,0.019676,NaN,NaN,NaN,NaN,295641952,201911,CONTINUA
499997,295641952,201912,1,0,1,61,253,1284.34,58388.02,1823.62,...,1.0,0.001284,0.018416,NaN,NaN,NaN,NaN,295641952,201912,CONTINUA
499998,295641952,202001,1,0,1,61,254,11683.42,58697.96,13874.51,...,1.0,0.000876,0.018416,NaN,NaN,NaN,NaN,295641952,202001,CONTINUA


In [29]:
data['check'] = data.numero_de_cliente == data.numero_de_cliente_1

In [31]:
data.columns

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'tc_proporcion_consumo_dolares',
       'tc_proporcion_consumo_total_limite_total_cubierto',
       'tc_proporcion_pago_pesos', 'tc_proporcion_pago_dolares',
       'tc_proporcion_adelanto_pesos', 'tc_proporcion_adelanto_dolares',
       'numero_de_cliente_1', 'foto_mes_1', 'clase_ternaria_1', 'check'],
      dtype='object', length=239)